# Project: Movie Recommender System

In [ ]:
import os
os.getcwd()



'c:\\Users\\valar\\Documents\\Flatiron\\Assignments\\Phase 4\\Project\\Phase-4-Project---Movie-Recommendation-System\\notebook'

In [4]:
os.chdir(r'c:\Users\valar\Documents\Flatiron\Assignments\Phase 4\Project\Phase-4-Project---Movie-Recommendation-System')

In [5]:
%pwd


'c:\\Users\\valar\\Documents\\Flatiron\\Assignments\\Phase 4\\Project\\Phase-4-Project---Movie-Recommendation-System'

In [6]:
# load the dataset
import pandas as pd
import numpy as np

df = pd.read_csv('./data/ratings.csv')
df.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [8]:
# dropping the timestamp column as we don't need it
df = df.drop(['timestamp'],axis=1)

In [10]:
# checking that there are no null values in our dataset
df.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [9]:
# changing out df into the surprise 
# importing surprise
from surprise import Reader,Dataset
reader = Reader()
df_surprise = Dataset.load_from_df(df,reader)

### Doing neighbour-based similarity models
Looking at how many user's vs items we have. 

In [11]:
dataset = df_surprise.build_full_trainset()
print('Number of users',dataset.n_users,'\n')
print('Number of items',dataset.n_items)

Number of users 610 

Number of items 9724


In [13]:
# importing necessary libraries
from surprise.model_selection import cross_validate,GridSearchCV
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNBaseline,KNNWithMeans,KNNBasic


# Evaluating Model with SVD

In [39]:
# 
param_grid = {'n_factors': [20,50,80],
              'reg_all':[0.02,0.05],
              'n_epochs':[15,20]}
# Instantiate GridsearchCv 
grid_svd = GridSearchCV(SVD,param_grid=param_grid,measures=['rmse'])

# fitting to surprise dataset
grid_svd.fit(df_surprise)


In [40]:
print('Best Parameters:', grid_svd.best_params)
print('Best Score', grid_svd.best_score)

Best Parameters: {'rmse': {'n_factors': 50, 'reg_all': 0.05, 'n_epochs': 20}}
Best Score {'rmse': 0.8690494238648709}


Comparing our SVD model with KNNBasic,KNNBaseline,and KNNwithMeans

In [29]:

# cross-validating with KNNBasic
Basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic = cross_validate(Basic,df_surprise,n_jobs=-1,measures=['rmse'])

In [30]:
Basic_trmse = np.mean(cv_knn_basic['test_rmse'])
print('Average test RMSE is:', Basic_trmse)

Average test RMSE is: 0.9719862050294903


KNNBaseline

In [32]:
Baseline = KNNBaseline(sim_options={'name':"pearson",'user_based': True})
cv_knn_baseline = cross_validate(Baseline,df_surprise,n_jobs=-1,measures = ['rmse'])

Baseline_trmse = np.mean(cv_knn_baseline['test_rmse'])
print('Average test RMSE is:',Baseline_trmse )

Average test RMSE is: 0.8769036641395287


# KNNwithMeans

In [33]:
KwMeans = KNNWithMeans(sim_options={'name':"pearson",'user_based': True})
cv_knn_wMeans = cross_validate(KwMeans,df_surprise,n_jobs=-1,measures = ['rmse'])

kwMeans_trmse = np.mean(cv_knn_wMeans['test_rmse'])
print('Average test RMSE is:', kwMeans_trmse)

Average test RMSE is: 0.8972573346015127


In [34]:
Summary = [{'Model': 'SVD', 'Best RMSE': grid_svd.best_score},
           {'Model': 'KNNBasic', 'Best RMSE': Basic_trmse},
           {'Model': 'KNNBaseline', 'Best RMSE': Baseline_trmse},
           {'Model': 'KNNWithMeans', 'Best RMSE': kwMeans_trmse},]

df_summary = pd.DataFrame(Summary)
df_summary

,Model,Best RMSE
0,SVD,{'rmse': 0.8692750978758921}
1,KNNBasic,0.971986
2,KNNBaseline,0.876904
3,KNNWithMeans,0.897257


# Making Movie Recommendations with SVD
SVD had the lowest rmse score at 0.86. We are going to fit and make predictions on this model.

In [41]:
# Instantiat an SVD model
# from GridSearch, Best Params: {'n_factors': 50, 'reg_all': 0.05, 'n_epochs': 20}}

svd = SVD(n_factors=50,reg_all=0.05,n_epochs=20)
svd.fit(dataset)

In [42]:
# Example prediction
svd.predict(1,3)

Prediction(uid=1, iid=3, r_ui=None, est=3.9909502575141453, details={'was_impossible': False})

The estimated rating for user id 1, movieid 3 is 3.990 which is very close to its true rating = 4.0